In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path
from platform import system
from difflib import get_close_matches

import util
import lines
import group
import label
import records
import extract

In [30]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir)
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[-2]

indexes_path = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/tesseract_data_frames"
output_dir = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/extracted_indexes"

if system()=="Windows":
    indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
    output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

indexes_csv = util.list_files(indexes_path, suffix=".csv")

display(test_dfs, file)

['.././test_files/tesseract_data_frames/LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986_split.csv',
 '.././test_files/tesseract_data_frames/LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/LS_index_1920.csv']

'.././test_files/tesseract_data_frames/LS_index_1986.csv'

In [31]:
#pdf_df = util.ocr(file, start_page=1, save_to=".././test_files")
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)

lines_df

,line_text,artifact_text,x0,y0,x1,y1,page,dx
0,International Legislative Series,,224,236,2041,287,1,1817
1,Labour Office 1987—List No. 3,,225,289,2041,324,1,1816
2,LIST OF RECENT LEGISLATION,,587,404,1680,456,1,1093
3,This list indicates the more important enactme...,,288,561,2044,603,1,1756
4,noted in the official gazettes recently receiv...,,218,608,1729,651,1,1511
...,...,...,...,...,...,...,...,...
4687,(195) 89 1987—List No. 3,,243,264,2059,317,89,1816
4688,Spain—Uruguay,,244,405,561,448,89,317
4689,19/11/1986 Additional international technical ...,,248,474,2066,517,89,1818
4690,matters between the Republic of Uruguay and th...,,311,521,2066,565,89,1755


In [32]:
#rec_df = extract.extract_indexes(pdf_df, save_to=test_output_dir + os.path.basename(file))
rec_df = extract.extract_indexes(pdf_df)
#rec_df.to_csv("LS_index_1944.csv")
rec_df

,country,region,text,page,extracted_date,extracted_day,extracted_month,extracted_year
0,ALGERIA,,1/X/1985 Decree No. 85-243 laying down model r...,1,1/X/1985,1,X,1985
1,ALGERIA,,17/X11/1985 Decree No. 85-306 to establish a g...,1,17/X11/1985,17,X11,1985
2,ALGERIA,,"9/11/1986 National Charter. (ibid., 16 Feb.) T...",1,9/11/1986,9,11,1986
3,ALGERIA,,25/I1I/1986 Decree No. 86-60 determining the p...,1,25/I1I/1986,25,I1I,1986
4,ALGERIA,,30/1V/1986 Order concerning the provisions to ...,1,30/1V/1986,30,1V,1986
...,...,...,...,...,...,...,...,...
964,EEC,Norway—Switzerland,28/V1/1985 Agreement to supplement the Adminis...,88,28/V1/1985,28,V1,1985
965,EEC,Portugal—Spain,12/X1/1983 Agreement between Spain and Portuga...,88,12/X1/1983,12,X1,1983
966,EEC,Senegal—Zaire,26/11I/1986 Co-operation agreement between the...,88,26/11I/1986,26,11I,1986
967,EEC,Senegal—Zaire,(195) 89 1987—List No. 3,89,,,,


In [33]:
rec_df.loc[rec_df["page"]==10]

,country,region,text,page,extracted_date,extracted_day,extracted_month,extracted_year
93,BELGIUM,,1987—List No. 3 10 (116),10,,,,
94,BELGIUM,,14/11/1986 Act to amend Royal Order No. 179 of...,10,14/11/1986,14,11,1986
95,BELGIUM,,"19/11/1986 Royal Order, to amend the Royal Ord...",10,19/11/1986,19,11,1986
96,BELGIUM,,24/I11/1986 Royal Order amending the regulatio...,10,24/I11/1986,24,I11,1986
97,BELGIUM,,"7/IV/1986 Royal Order, to amend the Royal Orde...",10,7/IV/1986,7,IV,1986
98,BELGIUM,,28/V/1986 Royal Order amending the regulations...,10,28/V/1986,28,V,1986
99,BELGIUM,,31/VII/1986 Royal Order regarding works counci...,10,31/VII/1986,31,VII,1986
100,BELGIUM,,20/V111/1986 Royal Order regarding the provisi...,10,20/V111/1986,20,V111,1986
101,BELGIUM,,1/X/1986 Royal Order No. 465 extending the per...,10,1/X/1986,1,X,1986


In [34]:
a = rec_df.loc[209]["text"]
a

'4/V1/1986 Act respecting partial pensions. No. 346. (ibid.) These pensions are payable to persons between 60 and 67 years of age who reduce their hours of work. Entry into force: 1 Jan. 1987.] '

In [44]:
def extract_dates(rec_df):
    df = rec_df.copy()
    df["extracted_date"] = ""
    df["extracted_day"] = ""
    df["extracted_month"] = ""
    df["extracted_year"] = ""

    digits = "[0-9oOIltriSQzZ]"
    re_d1 = "^(([A-Za-z]{3}[.:,]{0,3}|[A-Za-z]{4}.?) ?([1-3IlzZr]?" + digits + ")(?![0-9])(st|nd|rd|fd|th)?)" # in the beginning, example: Nov. 4 | July 25th
    re_d2 = "^(([1-3IirltzZ]?" + digits + ")[/,;.]{1,2}([I1l]{0,3}[VX]?[I1l]{0,3})[/,;.]{1,2}" + digits + "{4})" # in the beginning, example: 13/III/1986 | 7/11/198S
    re_d3 = "(([1-3IirltzZr]?" + digits + ")(st|nd|rd|fd|th)? ?([A-Za-z]{3,})[,.]? ?([Ii1lrt]" + digits + "{3}))" # towards the end, example: 25th February, 1929
    dt = extract.get_date_type(df)
    re_d = None
    day_g = 0
    month_g = 0

    if dt > -1:
        if dt==1:
            re_d = re_d1
            day_g = 3
            month_g = 2
        if dt==2:
            re_d = re_d2
            day_g = 2
            month_g = 3
        if dt==3:
            re_d = re_d3
            day_g = 2
            month_g = 4

        for i, row in df.iterrows():
            text = re.sub("[\"'`“´‘]", "", row["text"])
            d = None
            
            if dt != 2:
                text = re.sub("[,.]", "", text)
            if dt != 3:
                d = re.search(re_d, text)
            if dt == 3:
                for m in re.finditer(re_d, text):
                    d = m

            if not d == None:
                df.loc[i, "extracted_date"] = d.group(1)
                df.loc[i, "extracted_day"] = d.group(day_g)
                df.loc[i, "extracted_month"] = d.group(month_g)

                if dt != 1:
                    y = re.search(digits + "{4}", d.group(1))
                    df.loc[i, "extracted_year"] = y.group()
                

    return df

In [125]:
df_r = extract_dates(rec_df)
df_r.tail(40)

,country,region,text,page,extracted_date,extracted_day,extracted_month,extracted_year
929,ZIMBABWE,,1985 Labour Relations (Workers’ Committees) (G...,83,,,,
930,ZIMBABWE,,1987—List No. 3 84 (190),84,,,,
931,ZIMBABWE,,30/V/1986 Public Service (Officers) (Misconduc...,84,30/V/1986,30,V,1986
932,ZIMBABWE,,20/V1/1986 Public Holidays and Prohibition of ...,84,20/V1/1986,20,V1,1986
933,ZIMBABWE,,27/V1/1986 Emergency Powers (Control of Income...,84,27/V1/1986,27,V1,1986
934,ZIMBABWE,,1/VII/1986 Labour Relations (Specification of ...,84,1/VII/1986,1,VII,1986
935,ZIMBABWE,,18/VII/1986 Pensions Review (No. 14) Notice 19...,84,18/VII/1986,18,VII,1986
936,ZIMBABWE,Algeria—Iraq,29/V1/1985 Convention regarding co-operation b...,84,29/V1/1985,29,V1,1985
937,ZIMBABWE,Algeria—France,22/XI11/1985 First Codicil to the Agreement of...,84,22/XI11/1985,22,XI11,1985
938,ZIMBABWE,Algeria—France,(191) 85 1987—List No. 3,85,,,,


In [126]:
def correct_digit_recognition(dig):
    
    cor = re.sub("[Iiltr]", "1", dig)
    cor = re.sub("[Zz]", "2", cor)
    cor = re.sub("S", "5", cor)
    cor = re.sub("[OoQ]", "0", cor)
    
    if cor=="0":
        cor = "9"
    
    return cor

In [129]:
def norm_month(month, date_type, ignore_case=True):
    mon_l = list()
    month = str(month)
    
    if date_type == 1:
        mon_l = ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sep", "Oct", "Dec"]
        
    elif date_type == 2:
        mon_l = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX", "X", "XI", "XII"]
        month = re.sub("[1il]", "I", month)
        ignore_case = False
        
    elif date_type == 3:
        mon_l = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

    else:
        return None
    
    if (month == "") | (month == None):
        return None
    
    if ignore_case:
        mon_l = [x.lower() for x in mon_l]
    
    sim = get_close_matches(month, mon_l, n=1, cutoff=0.2)
    
    return mon_l.index(sim[0])+1

In [130]:
df = df_r.copy()

df.insert(3, "date", "")
df.insert(4, "year", "")

for i, row in df.iterrows():
    da = row["extracted_day"]
    mo = row["extracted_month"]
    ye = row["extracted_year"]
    
    day = correct_digit_recognition(da)
    year = correct_digit_recognition(ye)
    month = norm_month(mo, 2)
    
    if mo != "":
        #df.loc[i, "day"] = day
        #df.loc[i, "month"] = month
        df.loc[i, "year"] = year
        df.loc[i, "date"] = f"{day}.{month}."

df

,country,region,text,date,year,page,extracted_date,extracted_day,extracted_month,extracted_year
0,ALGERIA,,1/X/1985 Decree No. 85-243 laying down model r...,1.10.,1985,1,1/X/1985,1,X,1985
1,ALGERIA,,17/X11/1985 Decree No. 85-306 to establish a g...,17.12.,1985,1,17/X11/1985,17,X11,1985
2,ALGERIA,,"9/11/1986 National Charter. (ibid., 16 Feb.) T...",9.2.,1986,1,9/11/1986,9,11,1986
3,ALGERIA,,25/I1I/1986 Decree No. 86-60 determining the p...,25.3.,1986,1,25/I1I/1986,25,I1I,1986
4,ALGERIA,,30/1V/1986 Order concerning the provisions to ...,30.4.,1986,1,30/1V/1986,30,1V,1986
...,...,...,...,...,...,...,...,...,...,...
964,EEC,Norway—Switzerland,28/V1/1985 Agreement to supplement the Adminis...,28.6.,1985,88,28/V1/1985,28,V1,1985
965,EEC,Portugal—Spain,12/X1/1983 Agreement between Spain and Portuga...,12.11.,1983,88,12/X1/1983,12,X1,1983
966,EEC,Senegal—Zaire,26/11I/1986 Co-operation agreement between the...,26.3.,1986,88,26/11I/1986,26,11I,1986
967,EEC,Senegal—Zaire,(195) 89 1987—List No. 3,,,89,,,,


In [8]:
lines_df.loc[lines_df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx
528,NETHERLANDs—conitinued.,,272,156,811,189,9,539
529,Decree issuing as adininistrative regulations ...,,364,216,1814,301,9,1450
530,"gub- section (1), of the Labour Act, 1919. rot...",,455,256,1806,341,9,1351
531,Neth. 8 .,",",546,305,808,343,9,262
532,Decree issuing public administrative ‘regulati...,,364,308,1814,396,9,1450
...,...,...,...,...,...,...,...,...
593,22 (4). .. .. .. ..,—,498,3132,1807,3173,9,1309
594,Regulations for lithographic establishments. 2...,,369,3157,1815,3219,9,1446
595,4-22 (6)..,,463,3220,668,3262,9,205
596,Provisional Act respecting ‘works councils in ...,,368,3239,1819,3312,9,1451


In [8]:
d = lines_df.loc[lines_df["page"]==9]
d_x0 = group_lines(d, "x1")
d_x0["page"] = 9
d_x0

,x1,lines,last_x1,last_x1_mean,count,page
0,886,364,886,886,1,9
1,1453,363,1453,1453,1,9
2,528,359,528,528,1,9
3,"[1761, 1755]","[356, 368]","[1761, 1755]",1758.0,2,9
4,"[1629, 1629]","[352, 369]","[1629, 1629]",1629.0,2,9
5,974,349,974,974,1,9
6,1247,338,1247,1247,1,9
7,421,335,421,421,1,9
8,"[1702, 1699]","[334, 367]","[1702, 1699]",1700.5,2,9
9,"[353, 356]","[330, 353]","[353, 356]",354.5,2,9


In [20]:
r_x0 = get_relevant_x0_bins(d_x0, 3)
r_x0

,x0,lines,last_x0,last_x0_mean,count,page
4,"[92, 112, 89, 117, 90, 92, 90, 104, 94, 94, 108]","[324, 326, 330, 331, 335, 351, 353, 365, 366, ...","[94, 108]",101.0,11,9
3,"[137, 136, 134, 134, 136, 137, 149, 138, 138, ...","[325, 327, 336, 339, 345, 350, 354, 357, 360, ...","[120, 136]",128.0,11,9
2,"[391, 389, 385, 391, 391, 389, 389, 392, 391, ...","[328, 329, 332, 333, 334, 337, 338, 340, 341, ...","[393, 389]",391.0,23,9


In [21]:
lines_df.loc[r_x0.iloc[0]["lines"]]

,line_text,artifact_text,x0,y0,x1,y1,page,dx
324,NORWAY (continued),,92,400,692,455,9,600
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551
330,PANAMA,,89,796,353,840,9,264
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716
335,PORTUGAL,,90,1115,421,1169,9,331
351,"Tes ‘of 25th May, 1931, to set up the National...",,92,2092,1831,2160,9,1739
353,SWEDEN,,90,2248,356,2305,9,266
365,L.S. 1936 (Nor. 4); amendments: 1937 (Nor. 5);...,*,104,3008,1837,3093,9,1733
366,1930 (Nor. 2); amendments: 1932 (Nor. 4); 1933...,,94,3065,1834,3132,9,1740
367,(Nor. 2); 1937 (Nor. 1); 1941 (Nor. 1); 1942 (...,,94,3116,1699,3180,9,1605


In [7]:
df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
df = label.assign_labels(df, x0_n)

co_df, p_l, p_g = label.correct_x0_types(df, bins_x0, bins_x1, x0_n)
ind_df = label.assign_labels(co_df, x0_n)
ind_df = label.approve_correction(df, ind_df, p_l)

ind_df["new_label"] = ind_df["label"]
ind_df = label.improve_country_classification(ind_df)

ind_df = records.extract_records(ind_df)

In [22]:
df.loc[df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,0,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,1,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,0,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,1,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,2,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,2,2,middle
330,PANAMA,,89,796,353,840,9,264,0,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,0,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,2,2,middle


In [23]:
co_df.loc[co_df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,1,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,2,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,1,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,2,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,3,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,3,2,middle
330,PANAMA,,89,796,353,840,9,264,1,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,1,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,3,2,middle


In [26]:
co, p_l, p_g = correct_x0_types(df, bins_x0, bins_x1, x0_n)

Width median 1752.2894736842106
[(1658.9333333333332, 2), (1775.093023255814, 14), (1703.0959302325582, 15), (1709.0063795853268, 25), (1707.2626538987688, 39), (1828.5263157894738, 64), (1705.3108974358975, 69), (1726.8848484848486, 70), (1703.9268292682927, 77), (1728.189024390244, 78), (1702.4452690166975, 79), (1732.2894736842106, 81)]
Pages small width [2, 15, 25, 39, 69, 70, 77, 78, 79, 81]
Pages big width [14, 64]


In [28]:
co.loc[co["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,0,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,1,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,0,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,1,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,2,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,2,2,middle
330,PANAMA,,89,796,353,840,9,264,0,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,0,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,2,2,middle


In [9]:
ind_df.loc[ind_df["page"]==10]

,country,region,text,page
90,SWEDEN,,"on _ workers’ protection™, . (S.F,-p, 259)» - ...",10
91,SWEDEN,,"1931, respecting sick funds® <r (S.F. pp. 444-...",10
92,SWEDEN,,SWITZERLAND CONFEDERATION Jan. 400 Resolution ...,10
93,SWEDEN,,"1935 (Swe. 4); 1936 (Swe. 7, A); see also: 193...",10
94,SWEDEN,,"(Swe. 2, C). * Consolidated text: L.S. 1931 (S...",10
95,SWEDEN,,"(Swe. 2); 1937 (Swe. 3); 1938 (Swe. 4). — *""L....",10
96,SWEDEN,,"1 S, 1930 (Swe. 1); amendments: L.S. 1938 (Swe...",10
97,SWEDEN,,"1942 (Swe. 2, B). — “LS. 1940 (Swe. 1, A). 1S....",10
98,SWEDEN,,Bulletin of the International Labour Office (B...,10
99,SWEDEN,,"269-283, and Vol. XIV, 1919, pp. 205-209. — “L...",10


In [25]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 20)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df, p_l, p_g

In [17]:
for i, file in enumerate(indexes_csv):
    save_path = output_dir + os.sep + os.path.basename(file)
    extract.extract_indexes(pd.read_csv(file), save_to=save_path)

Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1976.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1977.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1978.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1979.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1981.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1982.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_Index_1983.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1983_Spanish.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1984.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_year

## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung FIXED
- Date-Type-Erkennung fehlerhaft
    - 1920 FIXED
- 1920 Seite 11 Uruguay
    - unvollständige Datums-Erkennung
    

## Fragen

